
# Tema – Random Forest (Clasificare & Regresie)
## Analiza polling-ului politic din Canada

**Cerință:**  
- Antrenarea modelelor DOAR pe datele anterioare datei **2025-01-13**  
- Testarea pe datele de după această dată  
- Compararea rezultatelor prezise cu cele reale


In [ ]:

import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score


## 1. Încărcarea și curățarea datelor

In [ ]:

data = pd.read_csv('canada_polling_data.csv')

data = data.drop(columns=['Others'])
data['Last date of polling'] = pd.to_datetime(
    data['Last date of polling'], format='%B %d, %Y', errors='coerce'
)

party_columns = ['CPC', 'LPC', 'NDP', 'BQ', 'PPC', 'GPC']
for col in party_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

data = data.sort_values('Last date of polling')
data.set_index('Last date of polling', inplace=True)

data.head()


## 2. Definirea variabilelor țintă

In [ ]:

data['Winner'] = data[party_columns].idxmax(axis=1)
data['Winner_Percentage'] = data[party_columns].max(axis=1)

X = data[party_columns].fillna(data[party_columns].mean())
y_class = data['Winner']
y_reg = data['Winner_Percentage']


## 3. Split temporal (înainte / după 2025-01-13)

In [ ]:

cutoff_date = '2025-01-13'

X_train = X[X.index < cutoff_date]
X_test = X[X.index >= cutoff_date]

y_class_train = y_class[y_class.index < cutoff_date]
y_class_test = y_class[y_class.index >= cutoff_date]

y_reg_train = y_reg[y_reg.index < cutoff_date]
y_reg_test = y_reg[y_reg.index >= cutoff_date]

len(X_train), len(X_test)


## 4. Antrenarea modelelor Random Forest

In [ ]:

rf_classifier = RandomForestClassifier(
    n_estimators=1000,
    random_state=42,
    max_depth=20,
    min_samples_split=10
)
rf_classifier.fit(X_train, y_class_train)

rf_regressor = RandomForestRegressor(
    n_estimators=500,
    random_state=42
)
rf_regressor.fit(X_train, y_reg_train)


## 5. Predicții și evaluare

In [ ]:

y_class_pred = rf_classifier.predict(X_test)
y_reg_pred = rf_regressor.predict(X_test)

print("=== CLASIFICARE ===")
print("Acuratețe:", accuracy_score(y_class_test, y_class_pred))
print(classification_report(y_class_test, y_class_pred))

print("\n=== REGRESIE ===")
print("MSE:", mean_squared_error(y_reg_test, y_reg_pred))
print("R²:", r2_score(y_reg_test, y_reg_pred))


## 6. Compararea rezultatelor reale cu cele prezise

In [ ]:

comparison = pd.DataFrame({
    'Winner_Real': y_class_test,
    'Winner_Predicted': y_class_pred,
    'Percentage_Real': y_reg_test,
    'Percentage_Predicted': y_reg_pred
})

comparison.head()


In [ ]:

comparison.to_csv('comparison_real_vs_predicted.csv')



## Concluzie
Modelele Random Forest au fost antrenate pe date istorice și evaluate pe date viitoare,
respectând structura temporală a problemei.
